# **OSHA publications**

In [1]:
import re
import requests
from bs4 import BeautifulSoup
!pip install pymupdf
import fitz  # PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 110.6 MB/s eta 0:00:00


**Generic scraping function**

Takes a given URL and extracts the main features or content from the page. This function is designed to be reusable for any page on the target website.


In [2]:
# Funzione per scaricare PDF dal link e estrapoliamo il testo

def scarica_e_leggi_pdf(pdf_url):
    response = requests.get(pdf_url, stream=True)
    if response.status_code != 200:
        print(f"Errore nel download del PDF: {response.status_code}")
        return None

    nome_file_pdf = 'osha_temp.pdf'
    with open(nome_file_pdf, 'wb') as f:
        f.write(response.content)

    with fitz.open(nome_file_pdf) as doc:
        full_text = " ".join(page.get_text() for page in doc)

    full_text = re.sub(r'\s+', ' ', full_text).strip()
    return full_text


In [3]:
# Funzione per estrazione metadati e testo PDF da una pagina di articolo OSHA

def scrape_osha_details(book_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    p = requests.get(book_url, headers=headers)
    page = BeautifulSoup(p.content, 'lxml')

    # Metadati principali
    title_elem = page.find('h1')
    title = title_elem.text.strip() if title_elem else 'N/A'

    keywords_elem = page.find('ul', class_='field__items')
    keywords = keywords_elem.text.strip().replace('\n', ', ') if keywords_elem else 'N/A'

    descr_blocks = page.find_all(id=re.compile('^tmgmt-'))
    descr = ' '.join(p.get_text(strip=True, separator=' ') for p in descr_blocks)
    descr = re.sub(r'\s+', ' ', descr.replace('\xa0', ' ')).strip()

    date_elem = page.find('p', class_='datetime-style')
    date = date_elem.text.strip() if date_elem else 'N/A'

    link_pdf = page.find('div', class_='download-pdf')
    pdf_url = link_pdf.find('a')['href'] if link_pdf else None

    # Estrazione testo PDF usando la seconda funzione
    testo_pdf = scarica_e_leggi_pdf(pdf_url) if pdf_url else None

    return {
        'titolo': title,
        'keywords': keywords,
        'descrizione': descr,
        'data': date,
        'pdf_url': pdf_url,
        'testo_pdf': testo_pdf
    }

In [4]:
url = 'https://osha.europa.eu/it/publications/tms-pros-programme-supporting-msds-prevention-health-and-social-care-sector'
info = scrape_osha_details(url)

print(info.keys())
print(info['titolo'])
print(info['testo_pdf'][:500])

dict_keys(['titolo', 'keywords', 'descrizione', 'data', 'pdf_url', 'testo_pdf'])
Il programma TMS Pros: sostegno alla prevenzione dei disturbi muscolo-scheletrici (DMS) nel settore dell’assistenza sanitaria e sociale
1 CASE STUDY THE TMS PROS PROGRAMME – SUPPORTING MSDS PREVENTION IN THE HEALTH AND SOCIAL CARE SECTOR Introduction Musculoskeletal disorders (MSDs) account for 88% of recognised occupational diseases in France (i.e. almost 45,000). In addition, every year there are 70,000 workplace accidents (lumbago) related to manual handling of loads with at least four days off work.1 In the health and social care (HeSCare) sector, 95% of occupational diseases are related to MSDs, and more than 2.3 million wo


**Function to filter publications by topic**

Implements a function that searches for and selects one of the pre-defined topics available on the site, returning the corresponding OSHA URL for that specific topic. The function is build in order to not be case-sensitive, so the user can enter it in any combination of upper or lower case letters. This function searches pre-defined topics in either Italian or English, returning the correct OSHA publications URL with the proper language path (/it or /en) based on the topic language provided. It accepts one or multiple topics

In [28]:
from typing import Iterable, List, Union
import difflib
import unicodedata

def link_tematica(topics):

    # Costruisce l'URL OSHA per una o più tematiche (IT/EN, case-insensitive).

    TOPICS = [
    {"id": "4640", "it": "agricoltura e silvicoltura", "en": "agriculture and forestry"},
    {"id": "105", "it": "assistenza sociosanitaria", "en": "health and social care"},
    {"id": "4644", "it": "digitalizzazione", "en": "digitalisation"},
    {"id": "173", "it": "disturbi muscoloscheletrici", "en": "musculoskeletal disorders"},
    {"id": "115", "it": "donne e ssl", "en": "women and osh"},
    {"id": "56", "en": "esener"},  # Solo EN, se non ha IT
    {"id": "137", "it": "horeca", "en": "horeca"},
    {"id": "166", "it": "invecchiamento e ssl", "en": "ageing and osh"},
    {"id": "4645", "it": "istruzione", "en": "education"},
    {"id": "4652", "it": "la leadership e la partecipazione dei lavoratori", "en": "leadership and worker participation"},
    {"id": "310908", "it": "l'economia circolare", "en": "circular economy"},
    {"id": "44", "it": "posti di lavoro verdi", "en": "green jobs"},
    {"id": "66", "it": "manutenzione", "en": "maintenance"},
    {"id": "126", "it": "microimprese", "en": "microenterprises"},
    {"id": "54", "it": "nanomateriali", "en": "nanomaterials"},
    {"id": "2967", "it": "oira", "en": "oira"},
    {"id": "43", "it": "prevenzione degli infortuni", "en": "accident prevention"},
    {"id": "51", "it": "rischi emergenti", "en": "emerging risks"},
    {"id": "171", "it": "rumore nei luoghi di lavoro", "en": "noise"},
    {"id": "32", "it": "sostanze pericolose", "en": "dangerous substances"},
    {"id": "62", "it": "ssl e i giovani", "en": "osh and young people"},
    {"id": "97", "it": "stress e rischi psicosociali", "en": "psychosocial risks and mental health"},
    {"id": "57", "it": "trasporti"},  # Solo IT
    {"id": "181", "it": "una buona ssl è un vantaggio dal punto di vista economico", "en": "good osh is good for business"},
    {"id": "73", "it": "valutazione dei rischi", "en": "risk assessment"},
    {"id": "61", "en": "work related diseases"},  # Solo EN
    ]

    if isinstance(topics, str):
        topics = [topics]
    if not topics:
        raise ValueError("Fornire almeno una tematica.")

    # Normalizza input
    norm_topics = []
    for t in topics:
        t = t.strip().lower()
        t = " ".join(t.split())
        t = unicodedata.normalize("NFKC", t).replace("’", "'")
        norm_topics.append(t)

    # Set unificato per suggerimenti (tutti i nomi IT+EN)
    all_names = set()
    for tp in TOPICS:
        if "it" in tp:
            all_names.add(tp["it"])
        if "en" in tp:
            all_names.add(tp["en"])

    # Mappa e rileva lingua
    it_ids, en_ids = [], []
    missing = []
    for raw, norm in zip(topics, norm_topics):
        matched = False
        for tp in TOPICS:
            if "it" in tp and norm == tp["it"]:
                it_ids.append(tp["id"])
                matched = True
                break
            elif "en" in tp and norm == tp["en"]:
                en_ids.append(tp["id"])
                matched = True
                break
        if not matched:
            simili = difflib.get_close_matches(norm, all_names, n=3, cutoff=0.6)
            hint = f" (forse: {', '.join(simili)})" if simili else ""
            missing.append(f"'{raw}' non trovato{hint}")

    if missing:
        raise ValueError("Tematica/e non riconosciuta/e: " + "; ".join(missing))

    if it_ids and en_ids:
        raise ValueError("Mescolato IT e EN: usa una sola lingua.")

    if not it_ids and not en_ids:
        raise ValueError("Nessuna tematica valida.")

    lang = "it" if it_ids else "en"
    tags = it_ids if lang == "it" else en_ids
    base = f"https://osha.europa.eu/{lang}/publications"

    # Params f[i]=facet_tags:ID
    params = [f"f%5B{i}%5D=facet_tags%3A{tag}" for i, tag in enumerate(tags)]
    query = "&".join(params)

    return f"{base}?{query}"


In [43]:
# Un topic in italiano

print(link_tematica("Digitalizzazione"))
# -> https://osha.europa.eu/it/publications?f%5B0%5D=facet_tags%3A4644

# Due topic in inglese

print(link_tematica(["Digitalisation", "Education"]))
# -> https://osha.europa.eu/en/publications?f%5B0%5D=facet_tags%3A4644&f%5B1%5D=facet_tags%3A4645

# Due topic in italiano

print(link_tematica(["digitalizzazione", "istruzione"]))
# -> https://osha.europa.eu/it/publications?f%5B0%5D=facet_tags%3A4644&f%5B1%5D=facet_tags%3A4645

https://osha.europa.eu/it/publications?f%5B0%5D=facet_tags%3A4644
https://osha.europa.eu/en/publications?f%5B0%5D=facet_tags%3A4644&f%5B1%5D=facet_tags%3A4645
https://osha.europa.eu/it/publications?f%5B0%5D=facet_tags%3A4644&f%5B1%5D=facet_tags%3A4645


In [33]:
# Esempio per testare funzioni successive

link_tematica("Digitalizzazione")
url_digital = link_tematica("Digitalizzazione")
url_digital

'https://osha.europa.eu/it/publications?f%5B0%5D=facet_tags%3A4644'

**Extracting publication links for a specific topic**

Once the URL for a selected topic is found, this function navigates through a specified number of pages and collects all the links to publications related to that topic.

In [49]:
import requests, re, time, random
from bs4 import BeautifulSoup
from urllib.parse import urlencode, urlparse, parse_qs, urlunparse, urljoin

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'it-IT,it;q=0.9,en;q=0.8',
    'Referer': 'https://osha.europa.eu/it/',
    'Sec-Ch-Ua': '"Google Chrome";v="129", "Chromium";v="129", "Not=A?Brand";v="24"'
}

def add_page(url, page):
    p = urlparse(url)
    q = parse_qs(p.query)
    q['page'] = [str(page)]
    return urlunparse(p._replace(query=urlencode(q, doseq=True)))

def get_osha_links(base_url, max_pages=3, pause=(1, 2)):
    # Funzione unificata: funziona costruita per funzionare sia per tematiche che per specifici search (fallback automatici sui selector)
    results, seen = [], set()
    domain = "https://osha.europa.eu"

    for page in range(max_pages):
        url = add_page(base_url, page)
        r = requests.get(url, headers=HEADERS, timeout=20)
        if r.status_code != 200:
            continue

        soup = BeautifulSoup(r.text, "lxml")

        # Selector unificati: priorita a search (revamp), poi tematica (views-row), poi globali
        links = (soup.select('div.views-view-grid .revamp-row h2 a[href]') or  # Search grid
                 soup.select('div.revamp-row h2 a[href]') or
                 soup.select('div.views-row h2 a[href]') or  # Per tematiche
                 soup.select('a[href*="/publications/"]'))  # Ultimo fallback (tutti i pub links)

        for a in links:
            href = urljoin(domain, a['href'])
            # Filtro: solo link con slug valido dopo /publications/
            if '/publications/' in href and re.search(r'/publications/[^\s/]+', href) and href not in seen:
                seen.add(href)
                title = a.get_text(strip=True)
                results.append({'title': title, 'url': href})

        time.sleep(random.uniform(*pause))

    return results

In [8]:
links = get_osha_links(url_digital, max_pages=2)

# Stampa solo conteggio e lista
print(f"Trovate {len(links)} pubblicazioni totali.\n")
for item in links:
    print(f"{item['title']}\n{item['url']}\n")

Trovate 10 pubblicazioni totali.

Premio per le buone pratiche 2023-2025 della campagna «Ambienti di lavoro sani e sicuri»
https://osha.europa.eu/it/publications/healthy-workplaces-good-practice-awards-2023-25

Austria: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
https://osha.europa.eu/it/publications/austria-osh-pulse-2025-mental-health-and-digitalisation-work

Belgio: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
https://osha.europa.eu/it/publications/belgium-osh-pulse-2025-mental-health-and-digitalisation-work

Bulgaria: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
https://osha.europa.eu/it/publications/bulgaria-osh-pulse-2025-mental-health-and-digitalisation-work

Cipro: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
https://osha.europa.eu/it/publications/cyprus-osh-pulse-2025-mental-health-and-digitalisation-work

Croazia: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di

**Saving scraped links to a JSON file**

After gathering the links for a given topic, the function scrapes each link to extract relevant information and then saves the collected data into a JSON file for further use or analysis.

In [9]:
import json
import os

def salva_in_json(record, filename):
    # Aggiunge un record a un file JSON (creandolo se non esiste)
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
    else:
        data = []

    data.append(record)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


In [10]:
import time   # per non essere bloccati dal server OSHA
import random

def scrape_osha_topic(base_url, max_pages=5, output_file='osha_risultati.json'):
    links = get_osha_links(base_url, max_pages=max_pages)
    print(f"Trovate {len(links)} pubblicazioni totali.\n")

    for i, item in enumerate(links, start=1):
        print(f"[{i}/{len(links)}] Estraggo: {item['title']}")
        try:
            dati = scrape_osha_details(item['url'])
            dati['url_pubblicazione'] = item['url']
            salva_in_json(dati, output_file)
            print(f" Salvato: {item['title']}")
        except Exception as e:
            print(f" Errore su {item['url']}: {e}")

        time.sleep(random.uniform(1, 3))  # pausa random 1-3 sec


In [11]:
# Scraping per tematica
scrape_osha_topic(url_digital, max_pages=2, output_file='osha_digitalizzazione.json')

Trovate 10 pubblicazioni totali.

[1/10] Estraggo: Premio per le buone pratiche 2023-2025 della campagna «Ambienti di lavoro sani e sicuri»
 Salvato: Premio per le buone pratiche 2023-2025 della campagna «Ambienti di lavoro sani e sicuri»
[2/10] Estraggo: Austria: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
 Salvato: Austria: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
[3/10] Estraggo: Belgio: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
 Salvato: Belgio: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
[4/10] Estraggo: Bulgaria: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
 Salvato: Bulgaria: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
[5/10] Estraggo: Cipro: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
 Salvato: Cipro: OSH Pulse 2025: salute mentale e digitalizzazione sul luogo di lavoro
[6/10] Estraggo: Croazia:

**Search function using custom keywords**

Allows users to generate an OSHA publications search URL using any keyword of interest or multi-word phrase, requiring an explicit language parameter to target the correct site section (/it or /en). The search is not case-sensitive.

In [39]:
from urllib.parse import quote_plus
import unicodedata

def build_osha_search_url(query: str, lang: str, sort: str = "field_publication_date") -> str:

    # Costruisce l'URL di ricerca per le pubblicazioni OSHA.

    if not isinstance(query, str) or not query.strip():
        raise ValueError("La query deve essere una stringa non vuota.")
    if lang not in {"it", "en"}:
        raise ValueError("Il parametro 'lang' deve essere 'it' oppure 'en'.")

    # Normalizzazione leggera: unicode, trim, lower, spazi multipli
    q = unicodedata.normalize("NFKC", query).strip().lower()
    q = " ".join(q.split())
    q = q.replace("’", "'")

    encoded_query = quote_plus(q)
    base_url = f"https://osha.europa.eu/{lang}/publications"
    return f"{base_url}?search_api_fulltext={encoded_query}&sort_by={sort}"


In [40]:
# Esempi

print(build_osha_search_url("risk assessment", lang="en"))
# -> https://osha.europa.eu/en/publications?search_api_fulltext=risk+assessment&sort_by=field_publication_date

print(build_osha_search_url("sicurezza sul lavoro", lang="it"))
# -> https://osha.europa.eu/it/publications?search_api_fulltext=sicurezza+su+lavoro&sort_by=field_publication_date

print(build_osha_search_url("Workplace   Safety  ", lang="en"))
# -> https://osha.europa.eu/en/publications?search_api_fulltext=workplace+safety&sort_by=field_publication_date



https://osha.europa.eu/en/publications?search_api_fulltext=risk+assessment&sort_by=field_publication_date
https://osha.europa.eu/it/publications?search_api_fulltext=sicurezza+sul+lavoro&sort_by=field_publication_date
https://osha.europa.eu/en/publications?search_api_fulltext=workplace+safety&sort_by=field_publication_date


In [46]:
#  Funzione che chiede all’utente sia la query sia la lingua per costruire l’URL di ricerca

def ask_user_and_build_search_url():
    query = input("Inserisci la parola o frase da cercare su OSHA: ").strip()
    lang = input("Specifica la lingua ('it' per italiano, 'en' per inglese): ").strip().lower()

    # Validazione semplice del parametro lingua
    while lang not in {"it", "en"}:
        print("Lingua non valida. Inserisci 'it' oppure 'en'.")
        lang = input("Specifica la lingua ('it' per italiano, 'en' per inglese): ").strip().lower()

    url = build_osha_search_url(query, lang=lang)
    print("URL di ricerca OSHA:", url)
    return url

In [48]:
# Esecuzione
search_url = ask_user_and_build_search_url()

Inserisci la parola o frase da cercare su OSHA: Ai e Lavoro
Specifica la lingua ('it' per italiano, 'en' per inglese): it
URL di ricerca OSHA: https://osha.europa.eu/it/publications?search_api_fulltext=ai+e+lavoro&sort_by=field_publication_date


**Scraping based on keyword search results**

Building on the keyword search, this function paginates through the resulting publication links for a defined number of pages, and then scrapes detailed data from each one as in the previous steps.

In [17]:
links = get_osha_links(search_url, max_pages=2)

# Stampa solo conteggio e lista (una volta!)
print(f"Trovate {len(links)} pubblicazioni totali.\n")
for item in links:
    print(f"{item['title']}\n{item['url']}\n")

Trovate 10 pubblicazioni totali.

«Per lavoratori del settore dell’assistenza socio-sanitaria senza disturbi muscolo-scheletrici»: una campagna di sensibilizzazione
https://osha.europa.eu/it/publications/health-and-social-care-workers-free-musculoskeletal-disorders-awareness-campaign

Card® per la movimentazione ergonomica del paziente: promozione di buone pratiche di lavoro nel settore sanitario
https://osha.europa.eu/it/publications/ergonomic-patient-handling-cardr-promoting-good-working-practices-healthcare-sector

Stabilire pratiche di lavoro ergonomiche e corrette per la schiena all’interno delle organizzazioni: il programma BGW Ergo-Coach
https://osha.europa.eu/it/publications/establishing-ergonomic-and-back-friendly-working-practices-organisations-bgw-ergo-coach-programme

Premio per le buone pratiche 2023-2025 della campagna «Ambienti di lavoro sani e sicuri»
https://osha.europa.eu/it/publications/healthy-workplaces-good-practice-awards-2023-25

Austria: OSH Pulse 2025: cambiam

In [18]:
scrape_osha_topic(search_url, max_pages=2, output_file='osha_ai_e_lavoro.json')

Trovate 10 pubblicazioni totali.

[1/10] Estraggo: «Per lavoratori del settore dell’assistenza socio-sanitaria senza disturbi muscolo-scheletrici»: una campagna di sensibilizzazione
 Salvato: «Per lavoratori del settore dell’assistenza socio-sanitaria senza disturbi muscolo-scheletrici»: una campagna di sensibilizzazione
[2/10] Estraggo: Card® per la movimentazione ergonomica del paziente: promozione di buone pratiche di lavoro nel settore sanitario
 Salvato: Card® per la movimentazione ergonomica del paziente: promozione di buone pratiche di lavoro nel settore sanitario
[3/10] Estraggo: Stabilire pratiche di lavoro ergonomiche e corrette per la schiena all’interno delle organizzazioni: il programma BGW Ergo-Coach
 Salvato: Stabilire pratiche di lavoro ergonomiche e corrette per la schiena all’interno delle organizzazioni: il programma BGW Ergo-Coach
[4/10] Estraggo: Premio per le buone pratiche 2023-2025 della campagna «Ambienti di lavoro sani e sicuri»
 Salvato: Premio per le buone p